In [1]:
import pandas as pd

In [2]:
cleaned_daily_BDI_df = pd.read_csv("../raw_data/clean_data/interpolated_daily_BDI.csv")
cleaned_weekly_CIP_df = pd.read_csv("../raw_data/clean_data/cleaned_weekly_CIP.csv")
Cleaned_El_Nino_df = pd.read_csv("../raw_data/clean_data/Cleaned_El_Nino.csv")
cleaned_ocean_land_temp_df = pd.read_csv("../raw_data/clean_data/cleaned_ocean_land_temp.csv")
cleaned_weekly_CSTEEL_df = pd.read_csv("../raw_data/clean_data/cleaned_weekly_CSTEEL.csv")

In [3]:
days = pd.date_range(start='01/01/1995', end='14/11/2022', freq='D')
days = days.to_frame(index=False, name='Date')

/tmp/ipykernel_9127/3172855226.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  days = pd.date_range(start='01/01/1995', end='14/11/2022', freq='D')


In [4]:
days["Date"] = pd.to_datetime(days["Date"], format="%Y-%m-%d")
days["week"] = days["Date"].dt.weekday

In [5]:
# remove saturday and sunday
days = days[days["week"] != 5]
days = days[days["week"] != 6]

In [6]:
cleaned_weekly_CSTEEL_df = cleaned_weekly_CSTEEL_df.drop("close", axis=1)
cleaned_weekly_CSTEEL_df = cleaned_weekly_CSTEEL_df.rename(columns={'time': 'Date', 'csteel': 'CSTEEL_price'})
cleaned_weekly_CSTEEL_df["Date"] = pd.to_datetime(cleaned_weekly_CSTEEL_df["Date"], format="%Y-%m-%d")

In [7]:
data_df = pd.merge(days ,cleaned_weekly_CSTEEL_df, on='Date', how='outer', indicator=True)

In [8]:
data_df["CSTEEL_price"] = data_df["CSTEEL_price"].interpolate(method="linear", limit_direction='both', axis=0)

In [9]:
Cleaned_El_Nino_df = Cleaned_El_Nino_df.rename(columns={'time': 'Date', 'Temperature': 'El_Nino_Temperature'})
Cleaned_El_Nino_df["Date"] = pd.to_datetime(Cleaned_El_Nino_df["Date"], format="%Y-%m-%d")
data_df = pd.merge(data_df ,Cleaned_El_Nino_df, on='Date', how='left', indicator='True')

In [10]:
cleaned_ocean_land_temp_df = cleaned_ocean_land_temp_df.rename(columns={'time': 'Date'})
cleaned_ocean_land_temp_df["Date"] = pd.to_datetime(cleaned_ocean_land_temp_df["Date"], format="%Y-%m-%d")
data_df = pd.merge(data_df ,cleaned_ocean_land_temp_df, on='Date', how='left')

In [11]:
cleaned_weekly_CIP_df = cleaned_weekly_CIP_df.rename(columns={'time': 'Date'})
cleaned_weekly_CIP_df["Date"] = pd.to_datetime(cleaned_weekly_CIP_df["Date"], format="%Y-%m-%d")
data_df = pd.merge(data_df ,cleaned_weekly_CIP_df, on='Date', how='left')

In [12]:
cleaned_daily_BDI_df = cleaned_daily_BDI_df.rename(columns={'Price': 'BDI_Price'})
cleaned_daily_BDI_df["Date"] = pd.to_datetime(cleaned_daily_BDI_df["Date"], format="%Y-%m-%d")

data_df = pd.merge(data_df, cleaned_daily_BDI_df, on='Date', how='left')

In [13]:
data_df = data_df.set_index("Date")

In [14]:
data_df = data_df[["BDI_Price", "CSTEEL_price", "El_Nino_Temperature", "Land Temperature", "Ocean Temperature", "CIP"]]

In [15]:
data_df["BDI_Price"] = data_df["BDI_Price"].interpolate(method="linear", limit_direction='both', axis=0)
data_df["El_Nino_Temperature"] = data_df["El_Nino_Temperature"].interpolate(method="linear", limit_direction='both', axis=0)
data_df["Land Temperature"] = data_df["Land Temperature"].interpolate(method="linear", limit_direction='both', axis=0)
data_df["Ocean Temperature"] = data_df["Ocean Temperature"].interpolate(method="linear", limit_direction='both', axis=0)
data_df["CIP"] = data_df["CIP"].interpolate(method="linear", limit_direction='both', axis=0)

In [16]:
data_df.head(20)

,BDI_Price,CSTEEL_price,El_Nino_Temperature,Land Temperature,Ocean Temperature,CIP
Date,,,,,,
1995-01-02,1964.0,7.380742e+06,0.770663,1.111744,0.384063,11.601218
1995-01-03,1964.0,7.380742e+06,0.770372,1.178854,0.387851,11.797236
1995-01-04,1961.0,7.380742e+06,0.769156,1.241437,0.391371,11.988118
1995-01-05,1967.0,7.380742e+06,0.767044,1.299601,0.394630,12.173925
1995-01-06,1983.0,7.380742e+06,0.764067,1.353453,0.397635,12.354720
1995-01-09,1994.0,7.380742e+06,0.750239,1.490200,0.405201,12.867653
1995-01-10,2015.0,7.380742e+06,0.744096,1.527870,0.407263,13.029021
1995-01-11,2019.0,7.380742e+06,0.737236,1.561761,0.409107,13.185688
1995-01-12,2038.0,7.380742e+06,0.729689,1.591981,0.410740,13.337717


In [17]:
data_df.to_csv("merged_daily_data.csv")